In [19]:
import pandas as pd

train = pd.read_excel(
    r"C:\Users\Mahi Sahu\OneDrive\Desktop\real_estate_multimodal\data\train.xlsx"
)


In [20]:
train['image_path'] = train['id'].apply(
    lambda x: rf"C:\Users\Mahi Sahu\OneDrive\Desktop\real_estate_multimodal\images\{x}.png"
)


In [21]:
train.head()


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,image_path
0,9117000170,20150505T000000,268643,4,2.25,1810,9240,2.0,0,0,...,1810,0,1961,0,98055,47.4362,-122.187,1660,9240,C:\Users\Mahi Sahu\OneDrive\Desktop\real_estat...
1,6700390210,20140708T000000,245000,3,2.50,1600,2788,2.0,0,0,...,1600,0,1992,0,98031,47.4034,-122.187,1720,3605,C:\Users\Mahi Sahu\OneDrive\Desktop\real_estat...
2,7212660540,20150115T000000,200000,4,2.50,1720,8638,2.0,0,0,...,1720,0,1994,0,98003,47.2704,-122.313,1870,7455,C:\Users\Mahi Sahu\OneDrive\Desktop\real_estat...
3,8562780200,20150427T000000,352499,2,2.25,1240,705,2.0,0,0,...,1150,90,2009,0,98027,47.5321,-122.073,1240,750,C:\Users\Mahi Sahu\OneDrive\Desktop\real_estat...
4,7760400350,20141205T000000,232000,3,2.00,1280,13356,1.0,0,0,...,1280,0,1994,0,98042,47.3715,-122.074,1590,8071,C:\Users\Mahi Sahu\OneDrive\Desktop\real_estat...


In [22]:
target_col = 'price'

tabular_cols = [
    col for col in train.columns 
    if col not in ['id', 'image_path', target_col]
]

X_tabular = train[tabular_cols]
y = train[target_col]


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image


ModuleNotFoundError: No module named 'torch'

In [6]:
import sys
sys.executable



'C:\\Users\\Mahi Sahu\\anaconda3\\python.exe'

In [7]:
import torch
torch.__version__


'2.9.1+cpu'

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image


In [23]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


In [25]:
class RealEstateDataset(Dataset):
    def __init__(self, df, tabular_cols, target_col, transform=None):
        self.df = df.reset_index(drop=True)
        self.tabular_cols = tabular_cols
        self.target_col = target_col
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img = Image.open(row['image_path']).convert('RGB')
        if self.transform:
            img = self.transform(img)

        tabular = torch.tensor(
           row[self.tabular_cols].astype("float32").values
     )


        target = torch.tensor(
            row[self.target_col], dtype=torch.float32
        )

        return img, tabular, target


In [26]:
dataset = RealEstateDataset(
    train,
    tabular_cols,
    target_col,
    transform=transform
)

loader = DataLoader(
    dataset,
    batch_size=16,
    shuffle=True
)


In [27]:
img, tab, y = next(iter(loader))
img.shape, tab.shape, y.shape


ValueError: could not convert string to float: '20140704T000000'

In [13]:
train[tabular_cols].dtypes


date              object
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [14]:
tabular_cols = train.select_dtypes(
    include=["int64", "float64"]
).columns.tolist()

tabular_cols.remove("price")  


In [15]:
train[tabular_cols] = train[tabular_cols].fillna(0)


In [28]:
img, tab, y = next(iter(loader))
img.shape, tab.shape, y.shape


ValueError: could not convert string to float: '20150227T000000'

In [29]:
train[tabular_cols].applymap(type).nunique()


C:\Users\Mahi Sahu\AppData\Local\Temp\ipykernel_16956\2966447475.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train[tabular_cols].applymap(type).nunique()


date             1
bedrooms         1
bathrooms        1
sqft_living      1
sqft_lot         1
floors           1
waterfront       1
view             1
condition        1
grade            1
sqft_above       1
sqft_basement    1
yr_built         1
yr_renovated     1
zipcode          1
lat              1
long             1
sqft_living15    1
sqft_lot15       1
dtype: int64

In [30]:
tabular_cols = [
    "bedrooms",
    "bathrooms",
    "sqft_living",
    "sqft_lot",
    "floors",
    "waterfront",
    "view",
    "condition",
    "grade",
    "sqft_above",
    "sqft_basement",
    "yr_built",
    "yr_renovated",
    "lat",
    "long",
    "sqft_living15",
    "sqft_lot15"
]


In [31]:
train[tabular_cols] = train[tabular_cols].apply(
    pd.to_numeric, errors="coerce"
).fillna(0)


In [32]:
dataset = RealEstateDataset(
    train,
    tabular_cols=tabular_cols,
    target_col="price",
    transform=transform
)

loader = DataLoader(dataset, batch_size=16, shuffle=True)


In [33]:
img, tab, y = next(iter(loader))
img.shape, tab.shape, y.shape


(torch.Size([16, 3, 224, 224]), torch.Size([16, 17]), torch.Size([16]))

In [34]:
import torch
import torch.nn as nn
import torchvision.models as models


In [37]:
cnn = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
cnn.fc = nn.Identity() 


In [38]:
class TabularNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )

    def forward(self, x):
        return self.net(x)


In [39]:
class MultiModalModel(nn.Module):
    def __init__(self, tabular_dim):
        super().__init__()
        self.cnn = cnn
        self.tabular = TabularNet(tabular_dim)

        self.regressor = nn.Sequential(
            nn.Linear(512 + 64, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, img, tab):
        img_feat = self.cnn(img)
        tab_feat = self.tabular(tab)
        fused = torch.cat([img_feat, tab_feat], dim=1)
        return self.regressor(fused).squeeze(1)


In [40]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = MultiModalModel(tabular_dim=len(tabular_cols))
model.to(device)


MultiModalModel(
  (cnn): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [41]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [42]:
img, tab, y = next(iter(loader))
img, tab, y = img.to(device), tab.to(device), y.to(device)

pred = model(img, tab)
pred.shape


torch.Size([16])

In [ ]:
import torch.optim as optim
import torch.nn as nn

model.train()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

epochs = 10  # or more depending on dataset size

for epoch in range(epochs):
    running_loss = 0.0
    for img, tab, y in loader:
        optimizer.zero_grad()
        output = model(img.float(), tab.float())
        loss = criterion(output.flatten(), y.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(loader):.4f}")
